In [1]:
from pyspark.sql import SparkSession

In [2]:
LAKE = "/Users/shaydabanihashemi/data/lake/trips"

BRONZE = f"{LAKE}/bronze"
SILVER = f"{LAKE}/silver"
GOLD = f"{LAKE}/gold"

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("trips").getOrCreate()

trips = spark.read.option("header", "true").csv(f"{BRONZE}/poi.csv")

Error: LinkageError occurred while loading main class org.apache.spark.launcher.Main
	java.lang.UnsupportedClassVersionError: org/apache/spark/launcher/Main has been compiled by a more recent version of the Java Runtime (class file version 61.0), this version of the Java Runtime only recognizes class file versions up to 55.0
/Users/shaydabanihashemi/ws/trips/.venv/lib/python3.12/site-packages/pyspark/bin/spark-class: line 97: CMD: bad array subscript
head: illegal line count -- -1


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [4]:
trips.show()

NameError: name 'trips' is not defined

In [8]:
#### Pull Categories column from original dataframe, delimit, remove leading/trailing spaces
from pyspark.sql.functions import split

categories = trips.select("categories")
categories = categories.withColumn("categories", split(categories["categories"], ";"))
categories.show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|categories                                                                                                                                                                                                                                                                                                                                                                                                       |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [9]:
cats = []
for row in categories.collect():
    if row["categories"] is not None:
        num_of_categories = len(row["categories"])
        for length in range(num_of_categories):
            cats.append(row["categories"][length].strip(' '))
print(f"Total Number of Categories: {len(cats)}")
print(f"Unique Number of Categories: {len(set(cats))}")
cats = list(dict.fromkeys(cats)) #I tried using set here - but ran into issues later

Total Number of Categories: 1170558
Unique Number of Categories: 157416


In [10]:
cat_id = []
for id in range(len(cats)):
    next_id = ("C-" + str(id))
    cat_id.append(next_id)
print(cat_id)

['C-0', 'C-1', 'C-2', 'C-3', 'C-4', 'C-5', 'C-6', 'C-7', 'C-8', 'C-9', 'C-10', 'C-11', 'C-12', 'C-13', 'C-14', 'C-15', 'C-16', 'C-17', 'C-18', 'C-19', 'C-20', 'C-21', 'C-22', 'C-23', 'C-24', 'C-25', 'C-26', 'C-27', 'C-28', 'C-29', 'C-30', 'C-31', 'C-32', 'C-33', 'C-34', 'C-35', 'C-36', 'C-37', 'C-38', 'C-39', 'C-40', 'C-41', 'C-42', 'C-43', 'C-44', 'C-45', 'C-46', 'C-47', 'C-48', 'C-49', 'C-50', 'C-51', 'C-52', 'C-53', 'C-54', 'C-55', 'C-56', 'C-57', 'C-58', 'C-59', 'C-60', 'C-61', 'C-62', 'C-63', 'C-64', 'C-65', 'C-66', 'C-67', 'C-68', 'C-69', 'C-70', 'C-71', 'C-72', 'C-73', 'C-74', 'C-75', 'C-76', 'C-77', 'C-78', 'C-79', 'C-80', 'C-81', 'C-82', 'C-83', 'C-84', 'C-85', 'C-86', 'C-87', 'C-88', 'C-89', 'C-90', 'C-91', 'C-92', 'C-93', 'C-94', 'C-95', 'C-96', 'C-97', 'C-98', 'C-99', 'C-100', 'C-101', 'C-102', 'C-103', 'C-104', 'C-105', 'C-106', 'C-107', 'C-108', 'C-109', 'C-110', 'C-111', 'C-112', 'C-113', 'C-114', 'C-115', 'C-116', 'C-117', 'C-118', 'C-119', 'C-120', 'C-121', 'C-122', 'C

In [11]:
#### Create new dataframe with category ID (generated) and category
from pyspark.sql.types import StructType, StructField, StringType

schema = StructType([
    StructField("category", StringType(), True),
    StructField("category_id", StringType(), True)])

x = 0
cats_structure = [("C-"+ str(x), cats[x]) for x in range(len(cats))]
complete_categories = spark.createDataFrame(cats_structure, schema)
complete_categories.show(truncate=False)

+--------+----------------------------------------------------------------------+
|category|category_id                                                           |
+--------+----------------------------------------------------------------------+
|C-0     |POPULATED PLACES IN CHALDORAN COUNTY                                  |
|C-1     |GLACIERS OF THE SIERRA NEVADA (U.S.)                                  |
|C-2     |GLACIERS OF FRESNO COUNTY, CALIFORNIA                                 |
|C-3     |GLACIERS OF CALIFORNIA                                                |
|C-4     |POPULATED PLACES IN SÉGOU REGION                                      |
|C-5     |IRRIGATION PROJECTS                                                   |
|C-6     |PEMBA ISLAND                                                          |
|C-7     |ISLANDS OF TANZANIA                                                   |
|C-8     |ISLANDS OF ZANZIBAR                                                   |
|C-9     |SWAHIL

In [12]:
links = trips.select("links")
links = links.withColumn("links", split(links["links"], ";"))
links.show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|links                                                                                                                                                                       

In [13]:
links_list = []
for row in links.collect():
    if row["links"] is not None:
        num_of_links = len(row["links"])
        for length in range(num_of_links):
            links_list.append(row["links"][length].strip(' '))
print(f"Total Number of Categories: {len(links_list)}")
print(f"Unique Number of Categories: {len(set(links_list))}")
links = list(dict.fromkeys(links_list)) #I tried using set here - but ran into issues later

Total Number of Categories: 8545275
Unique Number of Categories: 1422446


In [14]:
#### Create new dataframe with category ID (generated) and category
from pyspark.sql.types import StructType, StructField, StringType

schema = StructType([
    StructField("links", StringType(), True),
    StructField("links_id", StringType(), True)])

x = 0
links_structure = [("L-"+ str(x), links[x]) for x in range(len(links))]
complete_links = spark.createDataFrame(links_structure, schema)
complete_links.show(truncate=False)

25/06/12 18:29:08 WARN TaskSetManager: Stage 8 contains a task of very large size (5892 KiB). The maximum recommended task size is 1000 KiB.


+-----+-------------------------------------+
|links|links_id                             |
+-----+-------------------------------------+
|L-0  |Baba Jik Rural District              |
|L-1  |West Azerbaijan Province             |
|L-2  |Chaldoran County                     |
|L-3  |Romanize                             |
|L-4  |List of countries                    |
|L-5  |Counties of Iran                     |
|L-6  |Iran Daylight Time                   |
|L-7  |Iran Standard Time                   |
|L-8  |Provinces of Iran                    |
|L-9  |Rural Districts of Iran              |
|L-10 |Iran                                 |
|L-11 |Bakhsh                               |
|L-12 |Central District (Chaldoran County)  |
|L-13 |Mount Fiske                          |
|L-14 |Mount Warlow Glacier                 |
|L-15 |U.S. state                           |
|L-16 |List of glaciers in the United States|
|L-17 |United States                        |
|L-18 |Sierra Nevada (U.S.)       

25/06/12 18:29:12 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 8 (TID 22): Attempting to kill Python Worker


In [15]:
from pyspark.sql.functions import split
categories_links = trips.select("links", "categories")
list_categories_links = categories_links.withColumn("categories", split(categories_links["categories"], ";"))
list_categories_links = list_categories_links.withColumn("links", split(list_categories_links["links"], ";"))
list_categories_links.show(truncate=False)
#print(type(list_categories_links))

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [16]:
from pyspark.sql.functions import split
name_cats = trips.select("name", "categories")
list_name_cats = name_cats.withColumn("name", split(name_cats["name"], ";"))
list_name_cats = name_cats.withColumn("categories", split(name_cats["categories"], ";"))
list_name_cats.show(truncate=False)
#print(type(list_categories_links))

+--------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|name                                  |categories                                                                                                                                                                                                                                                                                                                                                                                                       |
+--------------------------------------+--------------------------------------------------------------------------

In [17]:
schema = StructType([
    StructField("name", StringType(), True),
    StructField("categories", StringType(), True)])
columns = ["name", "categories"]
new_row = []
for row in list_name_cats.collect():
    if row["name"] is not None and row["categories"] is not None:
        num_of_categories = len(row["categories"])
        for cat_length in range(len(row["categories"])):
            new_row.append((row["name"].strip(' '), row["categories"][cat_length].strip(' ')))
            #print(new_row)
try:
    complete_list_of_names_cats = spark.createDataFrame(new_row, schema)
except:
    print("9")
#complete_links = complete_links.union(new_row)
complete_list_of_names_cats.show(truncate=False)

25/06/12 18:29:18 WARN TaskSetManager: Stage 12 contains a task of very large size (7330 KiB). The maximum recommended task size is 1000 KiB.


+--------------------------------------+----------------------------------------------------------------------+
|name                                  |categories                                                            |
+--------------------------------------+----------------------------------------------------------------------+
|YAYCHI, WEST AZERBAIJAN               |POPULATED PLACES IN CHALDORAN COUNTY                                  |
|MOUNT FISKE GLACIER                   |GLACIERS OF THE SIERRA NEVADA (U.S.)                                  |
|MOUNT FISKE GLACIER                   |GLACIERS OF FRESNO COUNTY, CALIFORNIA                                 |
|MOUNT FISKE GLACIER                   |GLACIERS OF CALIFORNIA                                                |
|ALATONA                               |POPULATED PLACES IN SÉGOU REGION                                      |
|ALATONA                               |IRRIGATION PROJECTS                                             

25/06/12 18:29:22 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 12 (TID 33): Attempting to kill Python Worker


In [18]:
from pyspark.sql.functions import split
name_links = trips.select("name", "links")
list_name_links = name_links.withColumn("name", split(name_links["name"], ";"))
list_name_links = name_links.withColumn("links", split(name_links["links"], ";"))
list_name_links.show(truncate=False)
#print(type(list_categories_links))

+--------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|name                                  |links                                                                                         

In [19]:
schema_cat = StructType([
    StructField("name", StringType(), True),
    StructField("links", StringType(), True)])

new_row_links = []
for rows in list_name_links.collect():
    if rows["name"] is not None and rows["links"] is not None:
        num_of_links = len(rows["links"])
        for links_length in range(len(rows["links"])):
            new_row_links.append((rows["name"].strip(' '), rows["links"][links_length].strip(' ')))

try:
    complete_list_of_names_links = spark.createDataFrame(new_row_links, schema_cat)
except:
    print("9")
#complete_links = complete_links.union(new_row)
complete_list_of_names_links.show(truncate=False)

25/06/12 18:29:45 WARN TaskSetManager: Stage 15 contains a task of very large size (29628 KiB). The maximum recommended task size is 1000 KiB.


+-----------------------+-------------------------------------+
|name                   |links                                |
+-----------------------+-------------------------------------+
|YAYCHI, WEST AZERBAIJAN|Baba Jik Rural District              |
|YAYCHI, WEST AZERBAIJAN|West Azerbaijan Province             |
|YAYCHI, WEST AZERBAIJAN|Chaldoran County                     |
|YAYCHI, WEST AZERBAIJAN|Romanize                             |
|YAYCHI, WEST AZERBAIJAN|List of countries                    |
|YAYCHI, WEST AZERBAIJAN|Counties of Iran                     |
|YAYCHI, WEST AZERBAIJAN|Iran Daylight Time                   |
|YAYCHI, WEST AZERBAIJAN|Iran Standard Time                   |
|YAYCHI, WEST AZERBAIJAN|Provinces of Iran                    |
|YAYCHI, WEST AZERBAIJAN|Rural Districts of Iran              |
|YAYCHI, WEST AZERBAIJAN|Iran                                 |
|YAYCHI, WEST AZERBAIJAN|Bakhsh                               |
|YAYCHI, WEST AZERBAIJAN|Central Distric

25/06/12 18:29:49 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 15 (TID 43): Attempting to kill Python Worker


In [18]:
from trips.utilities import Speedy
import random
zips = (85255, 21087, 21234)
@Speedy
def lookup(location):
    return random.choice(zips)
print(lookup((34.052235, -118.243683)))
print(lookup((1, 2)))



85255
21087
